In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import random
import pandas as pd
import os
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

from mongodb import MongoAPI
from parser import PostParser # type: ignore
from parser import CommentParser # type: ignore

from crawler import PostCrawler
from crawler import CommentCrawler
import threading
import time
from datetime import datetime

In [2]:
post_df = pd.read_parquet("post_eastmoney_guba", filters=[('symbol', '==', "600519.SH"),])

In [13]:
options = webdriver.ChromeOptions()  # configure the webdriver
options.add_argument('lang=zh_CN.UTF-8')
options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                        'like Gecko) Chrome/111.0.0.0 Safari/537.36"')
options.add_argument('headless')
browser = webdriver.Chrome(options=options)

with open('stealth.min.js') as f:
    js = f.read()
browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": js
})

There was an error managing chromedriver (error sending request for url (https://googlechromelabs.github.io/chrome-for-testing/known-good-versions-with-downloads.json)); using driver found in the cache


{'identifier': '2'}

In [70]:

parser = CommentParser()

# for url in ["https://guba.eastmoney.com/news,600519,1427416892.html"]:
for url in ["https://guba.eastmoney.com/news,cfhpl,1426505054.html"]:
    if 'guba.eastmoney.com' not in url:
        continue
    try:
        time.sleep(abs(random.normalvariate(0.03, 0.01)))  # random sleep time

        try:  # sometimes the website needs to be refreshed (situation comment is loaded unsuccessfully)
            browser.get(url)  # this function may also get timeout exception
            WebDriverWait(browser, 0.2, poll_frequency=0.1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.reply_item.cl')))
        except TimeoutException:  # timeout situation
            browser.refresh()
            print('------------ refresh ------------')
        finally:
            reply_items = browser.find_elements(By.XPATH, '//*[@class="reply_item cl  "]')  # some have hot reply list avoid fetching twice

        # dic_list = []  # as batch insert is more efficient than insert one
        # for item in reply_items:
        #     dic = parser.parse_comment_info(item, id_df.iloc[self.current_num].item())
        #     # save the related post_id
        #     dic_list.append(dic)

        #     if parser.judge_sub_comment(item):  # means it has sub-comments
        #         sub_reply_items = item.find_elements(By.CSS_SELECTOR, 'li.reply_item_l2')

        #         for subitem in sub_reply_items:
        #             dic = parser.parse_comment_info(subitem, id_df.iloc[self.current_num].item(), True)
        #             # as it has sub-comments
        #             dic_list.append(dic)

    except TypeError as e:  # some comment is not allowed to display, just skip it
        1

In [73]:
reply_items = browser.find_elements(By.XPATH, '//div[@class="allReplyList"]/div[@class="replylist_content"]/div')

In [74]:
user_name = [i.find_element(By.XPATH, 'div/div[2]/a').text for i in reply_items]
user_id = [i.find_element(By.XPATH, 'div/div[2]/a').get_attribute("href").split("/")[-1] for i in reply_items]
reply_id = [i.get_attribute("data-reply_id") for i in reply_items]
reply_date = [i.find_element(By.XPATH, 'div/div[3]/span').text.split(" ")[0] for i in reply_items]
reply_time = [i.find_element(By.XPATH, 'div/div[3]/span').text.split(" ")[1] for i in reply_items]
replay_place = [i.find_element(By.XPATH, 'div/div[3]/span[2]').text.split(" ")[1] for i in reply_items]
reply_content = [i.find_element(By.XPATH, 'div/div[4]').text for i in reply_items]
reply_num = [i.find_element(By.XPATH, 'div/div[5]/ul/li/span[@class="replybtn"]').text for i in reply_items]
reply_num = [0 if i == '评论' else int(i) for i in reply_num]
like_num = [i.find_element(By.XPATH, 'div/div[5]/ul/li/span[@class="likemodule"]').text for i in reply_items]
like_num = [0 if i == '点赞' else int(i) for i in like_num]

In [75]:
sum(reply_num)

4

In [76]:
comment_dict_list = []

for pos in range(len(reply_id)):
    comment_dict = {
        "reply_id": reply_id[pos],
        "user_name":  user_name[pos],
        "user_id":  user_id[pos],
        "reply_date": reply_date[pos],
        "reply_time": reply_time[pos],
        "replay_place": replay_place[pos],
        "reply_content": reply_content[pos],
        "reply_num": reply_num[pos],
        "like_num": like_num[pos],
        'father_reply_id': "",
    }
    comment_dict_list.append(comment_dict)
    print(comment_dict)

{'reply_id': '9546993651', 'user_name': '活力的谢茁', 'user_id': '5096376473910202', 'reply_date': '2024-05-17', 'reply_time': '20:35:12', 'replay_place': '广东', 'reply_content': '打算买入018762，稳一手，债基还是比较稳定', 'reply_num': 0, 'like_num': 1, 'father_reply_id': ''}
{'reply_id': '9546933825', 'user_name': '基民Z7006992o3', 'user_id': '1534016921093216', 'reply_date': '2024-05-17', 'reply_time': '17:48:27', 'replay_place': '内蒙古', 'reply_content': '买了点018101，看好它的长期发展，希望不会让我失望', 'reply_num': 0, 'like_num': 0, 'father_reply_id': ''}
{'reply_id': '9546747690', 'user_name': '我是想试试名字到底可以取多长', 'user_id': '3636396417331580', 'reply_date': '2024-05-17', 'reply_time': '13:17:56', 'replay_place': '重庆', 'reply_content': '要选择正确的方向坚持呀，$南方亚太精选ETF发起联接C$ 应该是大A唯一第一大持仓是台积电的跨境etf了吧，成分股基本都是亚太区的半导体龙头，#胡锡进又加仓4万 A股是要反弹了吗？# 看好，蹲海外半导体周期', 'reply_num': 0, 'like_num': 2, 'father_reply_id': ''}
{'reply_id': '9546660275', 'user_name': '雪飘翼15', 'user_id': '6063057140254896', 'reply_date': '2024-05-17', 'reply_time': '10:59:28', 're

In [77]:
try:
    for item in browser.find_elements(By.XPATH, '//*[@class="moreL2reply"]/span[2]'):
        item.click()
except:
    for item in browser.find_elements(By.XPATH, '//*[@class="moreL2reply"]/span[2]'):
        item.click()

In [78]:
sub_reply = browser.find_elements(By.XPATH, '//div[@class="allReplyList"]/div/div/div/ul/li[@class="reply_item_l2  "]')

In [79]:
len(sub_reply)

4

In [80]:
user_name = [i.find_element(By.XPATH, 'div[@class="reuser_l2 cl"]/div/div[2]/a').text for i in sub_reply]
user_id = [i.find_element(By.XPATH, 'div[@class="reuser_l2 cl"]/div/div[2]/a').get_attribute("href").split("/")[-1] for i in sub_reply]
reply_id_s = [i.get_attribute("data-reply_id") for i in sub_reply]
reply_date = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[1]').text.split(" ")[0] for i in sub_reply]
reply_time = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[1]').text.split(" ")[1] for i in sub_reply]
replay_place = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[2]').text.split(" ")[1] for i in sub_reply]
reply_content = [i.find_element(By.XPATH, './/*[@class="reply_title_span"]').text for i in sub_reply]
reply_num_s = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/ul/li[2]').text for i in sub_reply]
reply_num_s = [0 if i == '评论' else int(i) for i in reply_num_s]
like_num = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/ul/li[3]').text for i in sub_reply]
like_num = [0 if i == '点赞' else int(i) for i in like_num]

In [82]:
pos = 0

for rpos in range(len(reply_num)):
    if reply_num[rpos] == 0:
        continue
    for i in range(reply_num[rpos]):
        f_rid = reply_id[rpos]
        comment_dict = {
            "reply_id": reply_id_s[pos],
            "user_name":  user_name[pos],
            "user_id":  user_id[pos],
            "reply_date": reply_date[pos],
            "reply_time": reply_time[pos],
            "replay_place": replay_place[pos],
            "reply_content": reply_content[pos],
            "reply_num": reply_num_s[pos],
            "like_num": like_num[pos],
            'father_reply_id': f_rid,
        }
        pos += 1
        comment_dict_list.append(comment_dict)

In [83]:
comment_dict_list

[{'reply_id': '9546993651',
  'user_name': '活力的谢茁',
  'user_id': '5096376473910202',
  'reply_date': '2024-05-17',
  'reply_time': '20:35:12',
  'replay_place': '广东',
  'reply_content': '打算买入018762，稳一手，债基还是比较稳定',
  'reply_num': 0,
  'like_num': 1,
  'father_reply_id': ''},
 {'reply_id': '9546933825',
  'user_name': '基民Z7006992o3',
  'user_id': '1534016921093216',
  'reply_date': '2024-05-17',
  'reply_time': '17:48:27',
  'replay_place': '内蒙古',
  'reply_content': '买了点018101，看好它的长期发展，希望不会让我失望',
  'reply_num': 0,
  'like_num': 0,
  'father_reply_id': ''},
 {'reply_id': '9546747690',
  'user_name': '我是想试试名字到底可以取多长',
  'user_id': '3636396417331580',
  'reply_date': '2024-05-17',
  'reply_time': '13:17:56',
  'replay_place': '重庆',
  'reply_content': '要选择正确的方向坚持呀，$南方亚太精选ETF发起联接C$ 应该是大A唯一第一大持仓是台积电的跨境etf了吧，成分股基本都是亚太区的半导体龙头，#胡锡进又加仓4万 A股是要反弹了吗？# 看好，蹲海外半导体周期',
  'reply_num': 0,
  'like_num': 2,
  'father_reply_id': ''},
 {'reply_id': '9546660275',
  'user_name': '雪飘翼15',
  'user_id': '60630571402

In [261]:
max_reply_page = browser.find_elements(By.XPATH, '//*[@class="pager"]/ul/li')
if max_reply_page != []:
    max_reply_page = int(max_reply_page[-2].text)

    for page in range(2, max_reply_page+1):
        max_reply_page

[]

In [85]:
[i.text for i in browser.find_elements(By.XPATH, '//*[@class="pager"]/ul/li')]

['1', '2', '']

In [3]:
comment_crawler = CommentCrawler('600519')


In [4]:
comment_crawler.find_by_date()

In [38]:
options = webdriver.ChromeOptions()  # configure the webdriver
options.add_argument('lang=zh_CN.UTF-8')
options.add_argument('user-agent="Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, '
                        'like Gecko) Chrome/111.0.0.0 Safari/537.36"')
options.add_argument('headless')
browser = webdriver.Chrome(options=options)

with open('stealth.min.js') as f:
    js = f.read()
browser.execute_cdp_cmd("Page.addScriptToEvaluateOnNewDocument", {
    "source": js
})

{'identifier': '2'}

In [18]:
try:  # sometimes the website needs to be refreshed (situation comment is loaded unsuccessfully)
    browser.get("https://guba.eastmoney.com/news,600519,1426089052_1.html")  # this function may also get timeout exception
    WebDriverWait(browser, 0.2, poll_frequency=0.1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.reply_item.cl')))
except TimeoutException:  # timeout situation
    browser.refresh()
    print('------------ refresh ------------')
finally:
    reply_items = browser.find_elements(By.XPATH, '//*[@class="reply_item cl  "]')  

------------ refresh ------------


In [19]:
html = browser
pid = 11
comment_dict_list = []

In [20]:

crawl_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
reply_items = html.find_elements(By.XPATH, '//*[@class="reply_item cl  "]')
user_name = [i.find_element(By.XPATH, 'div/div[2]/a').text for i in reply_items]
user_id = [i.find_element(By.XPATH, 'div/div[2]/a').get_attribute("href").split("/")[-1] for i in reply_items]
reply_id = [i.get_attribute("data-reply_id") for i in reply_items]
reply_date = [i.find_element(By.XPATH, 'div/div[3]/span').text.split(" ")[0] for i in reply_items]
reply_time = [i.find_element(By.XPATH, 'div/div[3]/span').text.split(" ")[1] for i in reply_items]
replay_place = [i.find_element(By.XPATH, 'div/div[3]/span[2]').text.split(" ")[1] for i in reply_items]
reply_content = [i.find_element(By.XPATH, 'div/div[4]').text for i in reply_items]
reply_num = [i.find_element(By.XPATH, 'div/div[5]/ul/li/span[@class="replybtn"]').text for i in reply_items]
reply_num = [0 if i == '评论' else int(i) for i in reply_num]
like_num = [i.find_element(By.XPATH, 'div/div[5]/ul/li/span[@class="likemodule"]').text for i in reply_items]
like_num = [0 if i == '点赞' else int(i) for i in like_num]

for pos in range(len(reply_id)):
    comment_dict = {
        "post_id": pid,
        "reply_id": reply_id[pos],
        "user_name":  user_name[pos],
        "user_id":  user_id[pos],
        "reply_date": reply_date[pos],
        "reply_time": reply_time[pos],
        "replay_place": replay_place[pos],
        "reply_content": reply_content[pos],
        "reply_num": reply_num[pos],
        "like_num": like_num[pos],
        'father_reply_id': "",
        'crawl_date': crawl_time.split(" ")[0],
        'crawl_time': crawl_time.split(" ")[1]
    }
    comment_dict_list.append(comment_dict)

sub_reply = html.find_elements(By.XPATH, '//li[@class="reply_item_l2  "]')
user_name = [i.find_element(By.XPATH, 'div[@class="reuser_l2 cl"]/div/div[2]/a').text for i in sub_reply]
user_id = [i.find_element(By.XPATH, 'div[@class="reuser_l2 cl"]/div/div[2]/a').get_attribute("href").split("/")[-1] for i in sub_reply]
reply_id_s = [i.get_attribute("data-reply_id") for i in sub_reply]
reply_date = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[1]').text.split(" ")[0] for i in sub_reply]
reply_time = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[1]').text.split(" ")[1] for i in sub_reply]
replay_place = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/div/span[2]').text.split(" ")[1] for i in sub_reply]
reply_content = [i.find_element(By.XPATH, './/*[@class="reply_title_span"]').text for i in sub_reply]
reply_num_s = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/ul/li[2]').text for i in sub_reply]
reply_num_s = [0 if i == '评论' else int(i) for i in reply_num]
like_num = [i.find_element(By.XPATH, 'div[@class="reply_bottom_l2 cl"]/ul/li[3]').text for i in sub_reply]
like_num = [0 if i == '点赞' else int(i) for i in like_num]

pos = 0
for rpos in range(len(reply_num)):
    if reply_num[rpos] == 0:
        continue
    for _ in range(reply_num[rpos]):
        f_rid = reply_id[rpos]
        comment_dict = {
            "post_id": pid,
            "reply_id": reply_id_s[pos],
            "user_name":  user_name[pos],
            "user_id":  user_id[pos],
            "reply_date": reply_date[pos],
            "reply_time": reply_time[pos],
            "replay_place": replay_place[pos],
            "reply_content": reply_content[pos],
            "reply_num": reply_num_s[pos],
            "like_num": like_num[pos],
            'father_reply_id': f_rid,
            'crawl_date': crawl_time.split(" ")[0],
            'crawl_time': crawl_time.split(" ")[1]
        }
        pos += 1
        comment_dict_list.append(comment_dict)

In [32]:
url = "https://guba.eastmoney.com/news,600519,1426089052.html"

In [40]:
parser = CommentParser()

In [41]:
try:  # sometimes the website needs to be refreshed (situation comment is loaded unsuccessfully)
    browser.get("https://guba.eastmoney.com/news,600519,1426089052.html")  # this function may also get timeout exception
    WebDriverWait(browser, 0.2, poll_frequency=0.1).until(
        EC.presence_of_element_located((By.CSS_SELECTOR, 'div.reply_item.cl')))
except TimeoutException:  # timeout situation
    browser.refresh()
finally:
    max_reply_page = browser.find_elements(By.XPATH, '//*[@class="pager"]/ul/li')
    if max_reply_page == []:
        max_reply_page = 1
    else:
        max_reply_page = int(max_reply_page[-2].text)
    for reply_page in range(1, max_reply_page+1):
        url_page = f"{url[:-5]}_{reply_page}.html"
        print(url_page)
        try:  # sometimes the website needs to be refreshed (situation comment is loaded unsuccessfully)
            browser.get(url_page)  # this function may also get timeout exception
            WebDriverWait(browser, 0.2, poll_frequency=0.1).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, 'div.reply_item.cl')))
        except TimeoutException:  # timeout situation
            browser.refresh()
            print('------------ refresh ------------')
        finally:
            parser.parse_comment_info(browser, pid)

https://guba.eastmoney.com/news,600519,1426089052_1.html


In [143]:
df.drop_duplicates(subset=['post_id'])

,post_id,post_title,stockbar_code,user_nickname,user_id,post_click_count,post_forward_count,post_comment_count,post_type,post_source_id,publish_time,update_date,update_time,crawl_date,crawl_time,post_url,symbol,publish_date
0,1426108245,可口可乐再创历史新高，茅台作为全球最优质的白酒公司，即将开启价值回归之旅[献花],600519,金银四海的傅红雪,7160395923239614,175,0,0,0,,23:54:45,2024-05-13,23:54:45,2024-05-17,22:28:58,"https://guba.eastmoney.com/news,600519,1426108...",600519.SH,2024-05-13
1,1426107652,[哭][哭][哭][哭],600519,股友903870Ph52,7471386341766762,107,0,0,0,,23:50:05,2024-05-13,23:50:05,2024-05-17,22:28:58,"https://guba.eastmoney.com/news,600519,1426107...",600519.SH,2024-05-13
2,1426102657,跌的不多，比想的好,cfhpl,beiyixiaohai,3365045001788924,1028,9,5,20,20240513231523463106070,23:15:23,2024-05-14,22:39:14,2024-05-17,22:28:58,"https://guba.eastmoney.com/news,cfhpl,14261026...",600519.SH,2024-05-13
3,1426102064,贵州茅台成交,600519,股友09660Q6I87,1407017007963536,79,0,0,0,,23:12:03,2024-05-13,23:12:03,2024-05-17,22:28:58,"https://guba.eastmoney.com/news,600519,1426102...",600519.SH,2024-05-13
4,1426097947,一季度收入增长18%，利润率受生产节奏扰动,600519,芯药酒合伙人,3708094327375356,326,0,1,20,20240513225134167827330,22:51:33,2024-05-14,13:59:44,2024-05-17,22:28:58,"https://guba.eastmoney.com/news,cfhpl,14260979...",600519.SH,2024-05-13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,1427394268,近30倍市赢之茅台和苹果再谈重大投资价值和投资性价比，可能就是一笑话而已？,600519,东方老山羊1234,1833094356192076,158,0,2,0,,06:27:56,2024-05-17,07:39:46,2024-05-17,22:28:56,"https://guba.eastmoney.com/news,600519,1427394...",600519.SH,2024-05-17
396,1427393363,本月超级金股三态股份 2024年5月17日周五晨,600519,轩辕理财室,4580112952257402,108,0,0,0,,06:16:25,2024-05-17,06:16:25,2024-05-17,22:28:56,"https://guba.eastmoney.com/news,600519,1427393...",600519.SH,2024-05-17
397,1427389464,高收益债券，韵达债券的收益高于茅台,zssh000001,大石899,6969055368906806,179,0,8,20,20240517050228878621390,05:02:28,2024-05-17,08:01:43,2024-05-17,22:28:56,"https://guba.eastmoney.com/news,cfhpl,14273894...",600519.SH,2024-05-17
398,1427384253,史上第一大牛市已经启动了,zssh000001,股友5U33229o08,1335057008156274,7217,19,107,20,20240517021811173716210,02:18:11,2024-05-17,22:27:51,2024-05-17,22:28:56,"https://guba.eastmoney.com/news,cfhpl,14273842...",600519.SH,2024-05-17


In [133]:
x = df.drop_duplicates(subset=["post_id", "post_title", "stockbar_code"]).groupby('post_id').count()

In [137]:
df[df['post_id']==1427593416]

,post_id,post_title,stockbar_code,user_nickname,user_id,post_click_count,post_forward_count,post_comment_count,post_type,post_source_id,publish_time,update_date,update_time,crawl_date,crawl_time,post_url,symbol,publish_date
1006,1427593416,低空经济：低空飞机的安全性？低空飞机是1000万还是500万一架？还是1个亿一架,600519,书仙策略,8978365858294302,231,3,4,0,,12:08:00,2024-05-17,16:50:55,2024-05-17,21:32:45,"https://guba.eastmoney.com/news,600519,1427593...",600519.SH,2024-05-17
1101,1427593416,低空经济：低空飞机的安全性？低空飞机是1000万还是500万一架？还是1个亿一架,600519,书仙策略,8978365858294302,232,3,4,0,,12:08:00,2024-05-17,16:50:55,2024-05-17,21:50:22,"https://guba.eastmoney.com/news,600519,1427593...",600519.SH,2024-05-17
1154,1427593416,低空经济：低空飞机的安全性？低空飞机是1000万还是500万一架？飞行员的身体素,600519,书仙策略,8978365858294302,73,0,0,0,,12:08:00,2024-05-17,12:08:00,2024-05-17,13:26:31,"https://guba.eastmoney.com/news,600519,1427593...",600519.SH,2024-05-17


In [130]:
x.duplicated(keep='first').sum()

0

In [116]:
df.drop_duplicates(subset=["post_id", "post_title", "stockbar_code"]).drop_duplicates(subset='post_id', keep=True)

ValueError: keep must be either "first", "last" or False

In [94]:
df['post_id'].drop_duplicates()

0       1426108245
1       1426107652
2       1426102657
3       1426102064
4       1426097947
           ...    
1046    1427394268
1047    1427393363
1048    1427389464
1049    1427384253
1050    1427384088
Name: post_id, Length: 442, dtype: int64

In [97]:
df = pd.read_parquet("comment_eastmoney_guba")

In [98]:
df[df['post_id']==1427374978]['reply_id'].drop_duplicates()

2653    9546899638
2654    9546738910
2655    9546727598
2656    9546723526
2657    9546723402
           ...    
2761    9546614949
2762    9546602717
2763    9546569935
2764    9546550725
2765    9546687047
Name: reply_id, Length: 113, dtype: object